<a href="https://colab.research.google.com/github/So-AI-love/CtrlRegen/blob/main/ctrlregen_plus_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download the code
!git clone https://github.com/yepengliu/CtrlRegen.git
%cd CtrlRegen

# install the required environment
! pip install -r requirements.txt

Cloning into 'CtrlRegen'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 131 (delta 49), reused 44 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (131/131), 1.16 MiB | 8.16 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/CtrlRegen
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.4/282.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
!pip install huggingface_hub==0.25.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.


In [3]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 19.8 MB/s eta 0:00:00


In [5]:
!pip install --upgrade pillow==6.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pillow: filename=Pillow-6.2.2-cp311-cp311-linux_x86_64.whl size=1073168 sha256=f884e8f6ede9645d69372c551e9dd74f9207575cbf672971fece19fb20d613d8
  Stored in directory: /root/.cache/pip/wheels/ea/d4/bc/90c63c126a7ecb0307a5832de074f3d57a7f73c519144bdd13
Successfully built pillow
  Attempting uninstall: pillow
    Found existing installation: pillow 10.3.0
    Uninstalling pillow-10.3.0:
      Successfully uninstalled pillow-10.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires Pillow>=7.0.0, but you have pillow 6.2.2 which is incompatible.
scikit-image 0.25.2 requires pillow>=10.1, but you have pillow 6.2.2 which is incompatible.
matplotlib 3.10.0 requires pillow>=8, but you have pillow 6.2.2 whic

In [4]:
%cd "/content/CtrlRegen"


import torch
from diffusers import ControlNetModel, UniPCMultistepScheduler, AutoencoderKL
from custom_i2i_pipeline import CustomStableDiffusionControlNetImg2ImgPipeline
from controlnet_aux import CannyDetector
from PIL import Image
import torchvision.transforms as transforms
from transformers import AutoModel, AutoImageProcessor
from utils import color_match

/content/CtrlRegen


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

ImportError: cannot import name 'is_directory' from 'PIL._util' (/usr/local/lib/python3.11/dist-packages/PIL/_util.py)

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

transform_size_to_512 = transforms.Compose([
        transforms.Resize(512, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(512),
        ])

DIFFUSION_MODEL = 'SG161222/Realistic_Vision_V4.0_noVAE'
SPATIAL_CONTROL_PATH = 'spatialnet_ckp/spatial_control_ckp_14000'
SEMANTIC_CONTROL_PATH = 'semanticnet_ckp'
SEMANTIC_CONTROL_NAME = 'semantic_control_ckp_435000.bin'
IMAGE_ENCODER = 'facebook/dinov2-giant'
VAE = 'stabilityai/sd-vae-ft-mse'

spatialnet = [ControlNetModel.from_pretrained(SPATIAL_CONTROL_PATH, torch_dtype=torch.float16)]
pipe = CustomStableDiffusionControlNetImg2ImgPipeline.from_pretrained(DIFFUSION_MODEL, \
                                                         controlnet=spatialnet, \
                                                         torch_dtype=torch.float16,
                                                         safety_checker = None,
                                                         requires_safety_checker = False
                                                         )
pipe.costum_load_ip_adapter(SEMANTIC_CONTROL_PATH, subfolder='models', weight_name=SEMANTIC_CONTROL_NAME)
pipe.image_encoder = AutoModel.from_pretrained(IMAGE_ENCODER).to(device, dtype=torch.float16)
pipe.feature_extractor = AutoImageProcessor.from_pretrained(IMAGE_ENCODER)
pipe.vae = AutoencoderKL.from_pretrained(VAE).to(dtype=torch.float16)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.set_ip_adapter_scale(1.0)
pipe.set_progress_bar_config(disable=True)
pipe.to(device)

processor = CannyDetector()

def ctrl_regen_plus(input_img, step, seed=0):
    generator = torch.manual_seed(seed)
    input_img = transform_size_to_512(input_img)
    processed_img = processor(input_img, low_threshold=100, high_threshold=150)
    prompt = 'best quality, high quality'
    negative_prompt = 'monochrome, lowres, bad anatomy, worst quality, low quality'
    output_img = pipe(prompt,
                      negative_prompt=negative_prompt,
                      image = [input_img],
                      control_image = [processed_img], # spatial condition
                      ip_adapter_image = [input_img],   # semantic condition
                      strength = step,
                      generator = generator,
                      num_inference_steps=50,
                      controlnet_conditioning_scale = 1.0,
                      guidance_scale = 2.0,
                      control_guidance_start = 0,
                      control_guidance_end = 1,
                      ).images[0]
    output_img = color_match(input_img, output_img)
    return output_img

In [ ]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("yepengliu/ctrlregen")

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
image = pipe(prompt).images[0]

In [ ]:
IMAGE_PATH = 'examples/1.jpg'
input_img = Image.open(IMAGE_PATH)
input_img

In [ ]:
%cd "/content/CtrlRegen"
import ctrl_regen_plus
IMAGE_PATH = 'examples/1.jpg'
input_img = Image.open(IMAGE_PATH)
# step could be between 0 and 1; change seed to get different regeneration results
output_img = ctrl_regen_plus(input_img=input_img, step=0.5, seed=1)
output_img